In [229]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation

import numpy as np
import random
import time

In [230]:
class VacumAgent(Agent):
    def __init__(self, id, model):
        # Inicializamos el agente con su identificador y el modelo al que pertenece
        super().__init__(id, model)

    # Esta función permite al agente moverse a una posición aleatoria en su vecindario
    def move(self):
        # Obtenemos todas las posiciones posibles a las que el agente puede moverse
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False)
        # Elegimos una posición aleatoria de las posiciones posibles
        new_position = random.choice(possible_steps)
        # Movemos al agente a la nueva posición
        self.model.grid.move_agent(self, new_position)

    # Esta función define el comportamiento del agente en cada paso de la simulación
    def step(self):
        # Si la posición actual del agente está sucia
        if self.model.is_dirt(self.pos):
            # El agente limpia la suciedad
            self.model.clean_dirt(self.pos)
            # Luego se mueve a una nueva posición
            self.move()
        else:
            # Si la posición actual no está sucia, el agente simplemente se mueve a una nueva posición
            self.move()

In [231]:
class VacumModel(Model):
    def __init__(self, width, height, num_agents, max_steps, dirty_cells):
        self.num_agents = num_agents
        self.max_steps = max_steps
        self.dirty_cells = dirty_cells
        self.grid = MultiGrid(width, height, torus=False)

        # creamos la matriz de ceros
        self.dirt = np.zeros((width, height))

        # Activamos de forma aleatoria en los agentes
        self.schedule = RandomActivation(self)

        self.steps = 0  # Contador de pasos

        # Calculamos el número de celdas sucias
        num_dirty_cells = int(dirty_cells * width * height)

        # Crea una lista de celdas sucias
        matrix = [(x, y) for x in range(width) for y in range(height)]
        random.shuffle(matrix)
        dirty_cells = matrix[:num_dirty_cells]
        for cell in dirty_cells:
            self.dirt[cell] = 1  # Marca las celdas sucias en la matriz

        # Inicializa nuestros agentes
        for i in range(num_agents):
            a = VacumAgent(i, self)  # Crea un nuevo agente
            # Coloca el agente en la posición (1,1)
            self.grid.place_agent(a, (1, 1))
            self.schedule.add(a)  # Añade el agente a la matriz

    def count_dirt(self):
        # Cuenta el número de celdas sucias
        return sum(cell == 1 for cell in self.dirt.flatten())

    def is_dirt(self, pos):
        # Comprueba si una celda está sucia
        return self.dirt[pos[0], pos[1]] == 1

    def clean_dirt(self, pos):
        # Limpia una celda
        self.dirt[pos[0]][pos[1]] = 0

    def step(self):
        # Realiza un paso en la simulación
        self.schedule.step()
        self.steps += 1  # Incrementa el contador de pasos

In [232]:
def run_model(grid_width, grid_height, agent_count, max_iterations, dirty_ratio):
    # Create an instance of the model with the provided parameters
    simulation_model = VacumModel(
        grid_width, grid_height, agent_count, max_iterations, dirty_ratio)

    # Initialize iteration counter
    iteration = 0

    # Run the simulation until all cells are clean or we reach the maximum number of iterations
    while iteration < max_iterations and np.any(simulation_model.dirt == 1):
        # On each iteration, every agent in the model performs an action
        simulation_model.step()

        # Increment the iteration counter
        iteration += 1

    # Return the number of iterations and the percentage of clean cells
    return (
        iteration,
        (1 - np.mean(simulation_model.dirt)) * 100
    )

In [233]:
WIDTH = 100
HEIGHT = 100
NUM_AGENTS = 10
MAX_STEPS = 1000000
DIRTY_CELLS = 0.9

start_time = time.time()
result = run_model(WIDTH, HEIGHT, NUM_AGENTS, MAX_STEPS, DIRTY_CELLS)
total_time = time.time() - start_time

print("Tiempo total de la ejecución: ", total_time)
print("Número de pasos realizados: " + str(result[0]))
print("Porcentaje de celdas limpias: " + str(result[1]) + "%")

Tiempo total de la ejecución:  2.1925082206726074
Número de pasos realizados: 28416
Porcentaje de celdas limpias: 100.0%
